In [1]:
import os
import sys
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)

In [2]:
import numpy as np
import pandas as pd
from vae_earlystopping import EarlyStopping
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from model.m26odel_mse import Modelmse
from model.m26odel_bce import Modelbce
from loss.l26oss_mse_1 import loss_mse
from loss.l26oss_bce_1 import loss_bce
import joblib
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
### 구하기
import json

In [ ]:


### Space Time 넣기  random_state
results = {
    "R2_React": [],      # bce_binary * mse (최종)
    "R2_Without_React": [],     # bce_prob * mse (sigmoid 가중)
}
for i in np.random.randint(1,100,size = 20):
    x_data = np.load('./data/metal.npy')
    c_data = np.load('./data/pre_re_fin.npy')
    
    x_train,x_test,c_train,c_test = train_test_split(x_data,c_data, random_state = i,test_size = 0.4)
    x_val,x_test,c_val,c_test = train_test_split(x_test,c_test,random_state = i, test_size = 0.5)
    x_scaler = MinMaxScaler()
    c_scaler = MinMaxScaler()
    x_train = x_scaler.fit_transform(x_train)
    c_train = c_scaler.fit_transform(c_train)
    x_val,x_test = [x_scaler.transform(x) for x in [x_val,x_test]]
    c_val,c_test = [c_scaler.transform(c) for c in [c_val,c_test]]

    x_train,x_val,x_test = [torch.tensor(x, dtype = torch.float32) for x in [x_train,x_val,x_test]]
    c_train,c_val,c_test = [torch.tensor(c, dtype = torch.float32) for c in [c_train,c_val,c_test]]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_data = [x_train,c_train]
    val_data = [x_val,c_val]
    test_data = [x_test,c_test]
    train_data = TensorDataset(*train_data)
    val_data = TensorDataset(*val_data)
    test_data = TensorDataset(*test_data)
    datas = [train_data,val_data,test_data]
    train_loader,val_loader,test_loader = [DataLoader(x,batch_size = 64,shuffle=False) for x in datas]
    x_sample, c_sample= next(iter(train_loader))

    x_dim = x_sample.shape[1]
    c_dim = c_sample.shape[1]

    model = Modelbce(x_dim,c_dim, z_dim=8).to(device)
    early_stopping = EarlyStopping(patience=40,min_delta = 1e-9)
    optimizer = optim.Adam(model.parameters(),lr = 1e-3, weight_decay=1e-5)
    epochs = 600
    for epoch in range(1,epochs+1):
        model.train()
        t_loss= 0
        for x, c in train_loader:
            x,c = x.to(device),c.to(device)
            optimizer.zero_grad()
            bce_logit, mu, logvar = model(x,c)
            loss_dict = loss_mse(bce_logit,x, mu,logvar)
            loss_dict['loss'].backward()
            optimizer.step()
            t_loss +=loss_dict['loss'].item()
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v_x, v_c in val_loader:
                v_x,v_c = v_x.to(device), v_c.to(device)
                v_bce_logit,v2_mu,v2_logvar = model(v_x,v_c)
                loss_dict = loss_bce(v_bce_logit, v_x, v2_mu,v2_logvar)
                v_loss += loss_dict['loss'].item()
        avg_train_loss = t_loss/len(train_loader)
        avg_val_loss = v_loss/len(val_loader)

        if epoch % 20 ==0  or epoch ==2:
            print(f'Epoch [{epoch}/{epochs}]|Train:{avg_train_loss:.4f} |Val:{avg_val_loss:.4f}')
        if early_stopping(avg_val_loss,model):
            break

    early_stopping.load_best_model(model)
    model.eval()
    all_bce = []
    with torch.no_grad():
        for x_t, c_t in test_loader:
            x_t,c_t = x_t.to(device),c_t.to(device)
            bce_logit_t,mu_t,logvar_t= model(x_t,c_t)
            probs = torch.sigmoid(bce_logit_t)
            all_bce.append(probs.cpu().numpy())
    bce_probs_total = np.vstack(all_bce)

    #mse
    model = Modelmse(x_dim,c_dim, z_dim=8).to(device)
    early_stopping = EarlyStopping(patience=40,min_delta = 1e-9)
    optimizer = optim.Adam(model.parameters(),lr = 1e-3, weight_decay=1e-5)
    epochs = 800
    for epoch in range(1,epochs+1):
        model.train()
        t_loss= 0
        for x,c in train_loader:
            x,c = x.to(device),c.to(device)
            optimizer.zero_grad()
            x_hat, mu, logvar = model(x,c)
            loss_dict = loss_mse(x_hat,x, mu,logvar)
            loss_dict['loss'].backward()
            optimizer.step()
            t_loss +=loss_dict['loss'].item()
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v_x, v_c in val_loader:
                v_x,v_c = v_x.to(device), v_c.to(device)
                v_bce_logit,v2_mu,v2_logvar = model(v_x,v_c)
                loss_dict = loss_mse(v_bce_logit, v_x, v2_mu,v2_logvar)
                v_loss += loss_dict['loss'].item()
        avg_train_loss = t_loss/len(train_loader)
        avg_val_loss = v_loss/len(val_loader)

        if epoch % 20 ==0  or epoch ==2:
            print(f'Epoch [{epoch}/{epochs}]|Train:{avg_train_loss:.4f} |Val:{avg_val_loss:.4f}')
        if early_stopping(avg_val_loss,model):
            break

    early_stopping.load_best_model(model)
    model.eval()
    all_x_hats = []
    all_x_trues = []
    with torch.no_grad():
        for x_t, c_t in test_loader:
            x_t,c_t = x_t.to(device),c_t.to(device)
            x_hat,mu_t,logvar_t= model(x_t,c_t)
            all_x_hats.append(x_hat.cpu().numpy())
            all_x_trues.append(x_t.cpu().numpy())
mse_logits_total = np.vstack(all_x_hats)  
x_true_total = np.vstack(all_x_trues)     
x_hat = mse_logits_total * bce_probs_total  
x_hat = x_scaler.inverse_transform(x_hat)
x_true= x_scaler.inverse_transform(x_true_total)

Epoch [2/600]|Train:0.1376 |Val:15.9586
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
Epoch [20/600]|Train:0.1170 |Val:15.9444
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [40/600]|Train:0.0980 |Val:15.9119
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
Ear

TypeError: loss_mse() missing 1 required positional argument: 'logvar'

In [ ]:
r2_bce_mse_sig = r2_score(x_true.flatten(), x_hat.flatten())
results["R2_React"].append(float(r2_bce_mse_sig))

In [ ]:
for i in np.random.randint(1,100,size = 20):
    x_data = np.load('./data/metal.npy')
    c_data = np.load('./data/pre_re_without_reac.npy')
    
    x_train,x_test,c_train,c_test = train_test_split(x_data,c_data, random_state = i,test_size = 0.4)
    x_val,x_test,c_val,c_test = train_test_split(x_test,c_test,random_state = i, test_size = 0.5)
    x_scaler = MinMaxScaler()
    c_scaler = MinMaxScaler()
    x_train = x_scaler.fit_transform(x_train)
    c_train = c_scaler.fit_transform(c_train)
    x_val,x_test = [x_scaler.transform(x) for x in [x_val,x_test]]
    c_val,c_test = [c_scaler.transform(c) for c in [c_val,c_test]]

    x_train,x_val,x_test = [torch.tensor(x, dtype = torch.float32) for x in [x_train,x_val,x_test]]
    c_train,c_val,c_test = [torch.tensor(c, dtype = torch.float32) for c in [c_train,c_val,c_test]]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_data = [x_train,c_train]
    val_data = [x_val,c_val]
    test_data = [x_test,c_test]
    train_data = TensorDataset(*train_data)
    val_data = TensorDataset(*val_data)
    test_data = TensorDataset(*test_data)
    datas = [train_data,val_data,test_data]
    train_loader,val_loader,test_loader = [DataLoader(x,batch_size = 64,shuffle=False) for x in datas]
    x_sample, c_sample= next(iter(train_loader))

    x_dim = x_sample.shape[1]
    c_dim = c_sample.shape[1]

    model = Modelbce(x_dim,c_dim, z_dim=8).to(device)
    early_stopping = EarlyStopping(patience=40,min_delta = 1e-9)
    optimizer = optim.Adam(model.parameters(),lr = 1e-3, weight_decay=1e-5)
    epochs = 600
    for epoch in range(1,epochs+1):
        model.train()
        t_loss= 0
        for x, c in train_loader:
            x,c = x.to(device),c.to(device)
            optimizer.zero_grad()
            bce_logit, mu, logvar = model(x,c)
            loss_dict = loss_mse(bce_logit,x, mu,logvar)
            loss_dict['loss'].backward()
            optimizer.step()
            t_loss +=loss_dict['loss'].item()
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v_x, v_c in val_loader:
                v_x,v_c = v_x.to(device), v_c.to(device)
                v_bce_logit,v2_mu,v2_logvar = model(v_x,v_c)
                loss_dict = loss_bce(v_bce_logit, v_x, v2_mu,v2_logvar)
                v_loss += loss_dict['loss'].item()
        avg_train_loss = t_loss/len(train_loader)
        avg_val_loss = v_loss/len(val_loader)

        if epoch % 20 ==0  or epoch ==2:
            print(f'Epoch [{epoch}/{epochs}]|Train:{avg_train_loss:.4f} |Val:{avg_val_loss:.4f}')
        if early_stopping(avg_val_loss,model):
            break

    early_stopping.load_best_model(model)
    model.eval()
    all_bce = []
    with torch.no_grad():
        for x_t, c_t in test_loader:
            x_t,c_t = x_t.to(device),c_t.to(device)
            bce_logit_t,mu_t,logvar_t= model(x_t,c_t)
            probs = torch.sigmoid(bce_logit_t)
            all_bce.append(probs.cpu().numpy())
    bce_probs_total = np.vstack(all_bce)

    #mse
    model = Modelmse(x_dim,c_dim, z_dim=8).to(device)
    early_stopping = EarlyStopping(patience=40,min_delta = 1e-9)
    optimizer = optim.Adam(model.parameters(),lr = 1e-3, weight_decay=1e-5)
    epochs = 800
    for epoch in range(1,epochs+1):
        model.train()
        t_loss= 0
        for x, c in train_loader:
            x,c = x.to(device),c.to(device)
            optimizer.zero_grad()
            x_hat, mu, logvar = model(x,c)
            loss_dict = loss_mse(x_hat,x, mu,logvar)
            loss_dict['loss'].backward()
            optimizer.step()
            t_loss +=loss_dict['loss'].item()
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v_x, v_c in val_loader:
                v_x,v_c = v_x.to(device), v_c.to(device)
                v_bce_logit,v2_mu,v2_logvar = model(v_x,v_c)
                loss_dict = loss_mse(v_bce_logit, v_x, v2_mu,v2_logvar)
                v_loss += loss_dict['loss'].item()
        avg_train_loss = t_loss/len(train_loader)
        avg_val_loss = v_loss/len(val_loader)

        if epoch % 20 ==0  or epoch ==2:
            print(f'Epoch [{epoch}/{epochs}]|Train:{avg_train_loss:.4f} |Val:{avg_val_loss:.4f}')
        if early_stopping(avg_val_loss,model):
            break

    early_stopping.load_best_model(model)
    model.eval()
    all_x_hats = []
    all_x_trues = []
    with torch.no_grad():
        for x_t, c_t in test_loader:
            x_t,c_t = x_t.to(device),c_t.to(device)
            x_hat,mu_t,logvar_t= model(x_t,c_t)
            all_x_hats.append(x_hat.cpu().numpy())
            all_x_trues.append(x_t.cpu().numpy())
mse_logits_total = np.vstack(all_x_hats)  
x_true_total = np.vstack(all_x_trues)     
x_hat = mse_logits_total * bce_probs_total  
x_hat = x_scaler.inverse_transform(x_hat)
x_true= x_scaler.inverse_transform(x_true_total)

Epoch [2/600]|Train:0.1091 |Val:15.9852
EarlyStopping counter: 1 out of 40


In [ ]:
r2_bce_mse_sig = r2_score(x_true.flatten(), x_hat.flatten())
results["R2_Without_React"].append(float(r2_bce_mse_sig))

In [ ]:
save_path = "./results_React_Const_Presence.json"
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Saved:", save_path)